# Génération documentation ontologie

In [2]:
import rdflib
from rdflib import RDF, OWL

In [63]:
g = rdflib.Graph()
result = g.parse("/home/mbunel/Documents/Ontologie/RelationsSpatiales/Relations_spatiales 8.owl")

In [142]:
import sys
import rdflib

class classDescriptor:
    
    def __init__(self, graph, uriClass):
        
        self.subClassUri = rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf')
        
        uriClassList = classes_rel_spa = list(g.subjects(self.subClassUri, uriClass))
        self.descriptor = { k:{'url':k} for k in uriClassList}
        
        for i in uriClassList:
            self.descriptor[i]['label'] = list(g.objects(i, rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label')))
            self.descriptor[i]['description'] = list(g.objects(i, rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#comment')))
        
    def out(self):
        
        lines = []
        for k,v in self.descriptor.items():
            for l in v['label']:
                lines.append("%s" % l.toPython())
            lines.append("/n")
            lines.append("/t%s/n" % v['url'])

            for d in v['description']:
                lines.append("%s/n" % d.toPython())
            lines.append("/n")
        return lines
    
        for l in lines:
            sys.stdout.write(l)
            
        sys.stdout.flush()
        sys.exit(0)
        
        
if __name__ == "__main__":
    
    path = sys.argv[0]
    
    g = rdflib.Graph()
    result = g.parse(path)
    
    tpe = rdflib.term.URIRef('http://www.semanticweb.org/mbunel/ontologies/Ornitho#Relation_spatiale')
    aa = classDescriptor(g, tpe)
    aa.out()

SAXParseException: file:///usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1:2: not well-formed (invalid token)

In [27]:
list(g.predicate_objects(rdflib.term.URIRef('http://www.semanticweb.org/mbunel/ontologies/Ornitho#DeAutreCoteDeParRapportA')))

[(rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class')),
 (rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'),
  rdflib.term.Literal("De l'autre côté de par rapport à", lang='fr')),
 (rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'),
  rdflib.term.URIRef('http://www.semanticweb.org/mbunel/ontologies/Ornitho#Relation_spatiale'))]

## Extraction classes

In [119]:
tpe = rdflib.term.URIRef('http://www.semanticweb.org/mbunel/ontologies/Ornitho#Relation_spatiale')

In [137]:
aa = classDescriptor(g, tpe)

In [138]:
aa.out()

["À l'est de",
 '/n',
 'http://www.semanticweb.org/mbunel/ontologies/Ornitho#AEstDe/n',
 'La cible se situe globalement à l’est du site, sans préciser si elle est à dans la partie est du site ou disjointe du site et à l’est de lui./n',
 '/n',
 'Au sud de',
 '/n',
 'http://www.semanticweb.org/mbunel/ontologies/Ornitho#AuSudDe/n',
 '/n',
 'À la même altitude que',
 '/n',
 'http://www.semanticweb.org/mbunel/ontologies/Ornitho#ALaMemeAltitudeQue/n',
 'La cible est située à la même altitude que le site. Le site peut être une altitude absolue (=> donc plus ou moins une courbe de niveau ou un «\xa0plan de niveau\xa0»), ou un objet dont l’altitude sert donc de référence./n',
 '/n',
 "Au dessus d'une altitude",
 '/n',
 'http://www.semanticweb.org/mbunel/ontologies/Ornitho#AuDessusDeAltitude/n',
 'La cible a une altitude supérieure à celle du site. La distance entre le site est la cible n’est pas contraignante./n',
 '/n',
 'Distance quantitative planimétrique',
 '/n',
 'http://www.semanticweb.or

In [109]:
test = rdflib.term.Literal("De l'autre côté de par rapport à", lang='fr')

In [111]:
test.toPython()

"De l'autre côté de par rapport à"